In [5]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
load_dotenv()
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

TRACELOOP_API_KEY= os.environ.get("TRACELOOP_API_KEY")

llm =ChatGroq(model ="llama-3.3-70b-versatile")

from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
set_llm_cache(InMemoryCache())


In [7]:
from langgraph.graph import MessagesState
def ChatBot(state:MessagesState):
    #Take user query
    query = state["messages"]
    
    #send to llm

    response = llm.invoke(query).content
    #store to the state

    return {"messages":[response]}

In [8]:
from langgraph.graph import StateGraph,START,END
from langgraph.checkpoint.memory import InMemorySaver
workflow =StateGraph(MessagesState)

#Add the nodes
workflow.add_node("chatbot",ChatBot)
# Add the edges
workflow.add_edge(START,"chatbot")
workflow.add_edge("chatbot",END)

checkpointer=InMemorySaver()
graph= workflow.compile(checkpointer=checkpointer)


In [ ]:
from langchain_core.runnables import RunnableConfig
import uuid
thread_id=uuid.uuid4()
config: RunnableConfig = {"configurable": {"thread_id": thread_id}}


res=graph.invoke({"messages":"I am amir ali"},config)
res

[HumanMessage(content='I am amir ali', additional_kwargs={}, response_metadata={}, id='2f472178-e8be-4628-bdbe-f2a6f38e4a3e'),
 HumanMessage(content="Hello Amir Ali, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={}, id='75ae0ba4-ada7-4b41-bc35-07fab96c3e03')]

In [13]:
res=graph.invoke({"messages":"What is My name ?"},config,durablility='async')
res

{'messages': [HumanMessage(content='I am amir ali', additional_kwargs={}, response_metadata={}, id='2f472178-e8be-4628-bdbe-f2a6f38e4a3e'),
  HumanMessage(content="Hello Amir Ali, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={}, id='75ae0ba4-ada7-4b41-bc35-07fab96c3e03'),
  HumanMessage(content='I am amir ali', additional_kwargs={}, response_metadata={}, id='b7989ec8-a08f-4949-b8ba-c6d4e5812fb4'),
  HumanMessage(content="Hello again, Amir Ali. It seems like you're introducing yourself. That's great! I'm happy to meet you. Is there something on your mind that you'd like to talk about, or perhaps a question you have? I'm all ears and here to help.", additional_kwargs={}, response_metadata={}, id='b49e8136-a56e-4fc1-813c-6c31348cd745'),
  HumanMessage(content='I am amir ali', additional_kwargs={}, response_metadata={}, id='a7a94fa7-0b9d-42e7-8986-9fa711320ffa'),
  HumanMessage(content="Hello Amir Ali, I 